# Chapter 10: Selecting and Creating Columns

In [ ]:
import polars as pl
pl.show_versions()  # The book is built with Polars version 1.13.1

In [ ]:
starwars = pl.read_parquet("data/starwars.parquet")
rebels = starwars.drop("films").filter(
    pl.col("name").is_in(["Luke Skywalker", "Leia Organa", "Han Solo"])
)

print(rebels[:, :6])  
print(rebels[:, 6:11])
print(rebels[:, 11:])

## Selecting Columns

In [ ]:
rebels.select(
    "name",
    pl.col("homeworld"),
    pl.col("^.*_color$"),
    (pl.col("height") / 100).alias("height_m"),
)

### Introducing Selectors

In [ ]:
import polars.selectors as cs

In [ ]:
rebels.select(
    "name",
    cs.by_name("homeworld"),
    cs.by_name("^.*_color$"),
    (cs.by_name("height") / 100).alias("height_m"),
)

### Selecting Based on Name

In [ ]:
rebels.select(cs.starts_with("birth_"))

In [ ]:
rebels.select(cs.ends_with("_color"))

In [ ]:
rebels.select(cs.contains("_"))

In [ ]:
rebels.select(cs.matches("^[a-z]{4}$"))

### Selecting Based on Data Type

In [ ]:
rebels.group_by("hair_color").agg(cs.numeric().mean())

In [ ]:
rebels.select(cs.string())

In [ ]:
rebels.select(cs.temporal())

In [ ]:
rebels.select(cs.by_dtype(pl.List(pl.String)))

### Selecting Based on Position

In [ ]:
rebels.select(cs.by_index(range(0, 999, 3)))  

In [ ]:
rebels.select("name", cs.by_index(range(-2, 0)))

In [ ]:
# This raises a ColumnNotFoundError:
# rebels.select(cs.by_index(20))

In [ ]:
rebels.select(cs.by_index(range(20, 22)))

### Combining Selectors

In [ ]:
rebels.select(cs.by_name("hair_color") | cs.numeric())

In [ ]:
df = pl.DataFrame({"d": 1, "i": True, "s": True, "c": True, "o": 1.0})

print(df)

x = cs.by_name("d", "i", "s")
y = cs.boolean()

print("\nselector => columns")

for s in ["x", "y", "x | y", "x & y", "x - y", "x ^ y", "~x", "x - x"]:
    print(f"{s:8} => {cs.expand_selector(df, eval(s))}")

In [ ]:
df.select(x - x)

In [ ]:
print(df.select(first := cs.by_name("c", "i"), ~first))
print(f"first: {first}, ~first: {~first}")

In [ ]:
print(df.select(first := cs.last(), ~first))
print(f"first: {first}, ~first: {~first}")

## Creating Columns

In [ ]:
rebels.with_columns(bmi=pl.col("mass") / ((pl.col("height") / 100) ** 2))

In [ ]:
df = pl.DataFrame({"a": [1, 2, 3]})
df.with_columns(pl.col("a") * 2)

In [ ]:
df.with_columns(a2=pl.col("a") * 2)

In [ ]:
from datetime import datetime

rebels.with_columns(
    bmi=pl.col("mass") / ((pl.col("height") / 100) ** 2),
    age_destroy=(
        (datetime(1983, 5, 25) - pl.col("birth_date")).dt.total_days() / 365
    ).cast(pl.UInt8),
)

In [ ]:
# This raises a ColumnNotFoundError:
# rebels.with_columns(
#     bmi=pl.col("mass") / ((pl.col("height") / 100) ** 2),
#     bmi_cat=pl.col("bmi").cut(
#         [18.5, 25], labels=["Underweight", "Normal", "Overweight"]
#     ),
# )

In [ ]:
(
    rebels.with_columns(
        bmi=pl.col("mass") / ((pl.col("height") / 100) ** 2)
    ).with_columns(
        bmi_cat=pl.col("bmi").cut(
            [18.5, 25], labels=["Underweight", "Normal", "Overweight"]
        )
    )
)

In [ ]:
# Causes a SyntaxError:
# starwars.select(
#     "name",
#     bmi=(pl.col("mass") / ((pl.col("height") / 100) ** 2)),
#     "species",
# )

In [ ]:
(
    starwars.select(
        "name",
        (pl.col("mass") / ((pl.col("height") / 100) ** 2)).alias("bmi"),  
        "species",
    )
    .drop_nulls()
    .top_k(5, by="bmi")  
)

In [ ]:
df.with_columns(pl.lit(1).alias("ones"))

In [ ]:
df.select(pl.all(), pl.lit(1).alias("ones"))

## Related Column Operations

In [ ]:
rebels.drop("name", "films", "screen_time", strict=False)  

In [ ]:
rebels.select(~cs.by_name("name", "films", "screen_time"))

In [ ]:
rebels.select(cs.exclude("name", "films", "screen_time"))

In [ ]:
(
    rebels.rename({"homeworld": "planet", "mass": "weight"})
    .rename(lambda s: s.removesuffix("_color"))
    .select("name", "planet", "weight", "hair", "skin", "eye")  
)

In [ ]:
rebel_names = rebels.select("name")
rebel_colors = rebels.select(cs.ends_with("_color"))
rebel_quotes = pl.Series(
    "quote",
    [
        "You know, sometimes I amaze myself.",
        "That doesn't sound too hard.",
        "I have a bad feeling about this.",
    ],
)

(rebel_names.hstack(rebel_colors).hstack([rebel_quotes]))  

In [ ]:
rebels.with_row_index(name="rebel_id", offset=1)

## Takeaways